<h1><center>  低质量内容检测 -  Nucleus API实例</center></h1>


<h1><center>  所有权及保密条款属SumUp Analytics所有</center></h1>


<h1><center>  免责声明和服务条款可通过 www.sumup.ai 获取 </center></h1>


 


## 目标: 
-	开发一条管道以检测社交媒体或游戏聊天室中的低质量内容

**在其当前版本中，SumUp对比度分析将两个类别相互比较，其中用户定义两个类别。**

## 数据:
-	来自社交媒体或游戏平台的标签语库
 -     您的语库中可以有多个标签，但在学习/预测时算法将是二进制分类
 
 
 - 用于低质量内容检测的说明性标签: 
  - "Violence" （暴力）
  - "Drugs"（毒品）
  - "Pornographic" （色情）
  - "Religiously Sensitive" （宗教敏感）
  - "Politically Sensitive" （政治敏感）
  - "Scam"（诈骗）
  - "Clickbait"
  - "Fake" （假）
  - "All clear"（全部清除）



## Nucleus APIs:
-	数据集创建  API
 - 	*api_instance.post_upload_file(file, dataset)*
 - 	*nucleus_helper.import_files(api_instance, dataset, file_iters, processes=1)*

        nucleus_helper.import_files利用api_instance.post_upload_file并行执行来加速数据集的创建


-	主题建模 API
 - 	*api_instance.post_topic_api(payload)*


-	对比主题建模 API
 - 	*api_instance.post_topic_contrast_api(payload)*


-	文件分类 API
 - 	*api_instance.post_doc_classify_api(payload)*

## 方法:

### 1.	数据集准备
-	创建一个Nucleus数据集，其中包含所选历史时期内的所有相关文档


-   在这种情况下，我们使用存储为csv文件的数据。 可以构建类似的代码以从数据库表中注入。 对传递给API以创建数据集的数据和元数据字段的名称有一些要求


    - 上传数据的模板: ["author", "label", "time", "content", "title"]

    

In [ ]:
import csv
import json
import nucleus_api.api.nucleus_api as nucleus_helper
import nucleus_api
from nucleus_api.rest import ApiException

configuration = nucleus_api.Configuration()
configuration.host = 'UPDATE-WITH-API-SERVER-HOSTNAME'
configuration.api_key['x-api-key'] = 'UPDATE-WITH-API-KEY'

# 创建API实例
api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

In [ ]:
csv_file = 'social-media.csv'
dataset = 'social-media'# str | Destination dataset where the file will be inserted.

with open(csv_file, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    json_props = nucleus_helper.upload_jsons(api_instance, dataset, reader, processes=4)
    
    total_size = 0
    total_jsons = 0
    for jp in json_props:
        total_size += jp.size
        total_jsons += 1
        
    print(total_jsons, 'JSON records (', total_size, 'bytes) appended to', dataset)


### 2. 对比主题建模

-     在这个例子中，我们定义和对比两个类别，即“Violence”(暴力)和“All clear”（清除）。


-     我们提取分隔这两个类别的对比主题（及其关键字）

In [ ]:
metadata_selection_contrast = {"label": ["Violence", "All clear"]} # dict | 元数据选择定义了两类文档，以便相互对比和总结
query = '' # str | Dataset-language-specific全文查询，使用mysql MATCH boolean query格式（可选）
custom_stop_words = [""] # 停用词列表。 （可选）
excluded_docs = '' # str | 应从分析中排除的文档ID列表。 例如，[“docid1”，“docid2”，...，“docidN”]（可选）
syntax_variables = False # bool | 指定是否考虑每类文档的语法方面以帮助对比它们（可选）（默认为False）
num_keywords = 200 # integer | 从数据集中提取的比较主题的关键字数。 (可选) (默认为50)
remove_redundancies = False # bool | 如果为True，则此选项从分析中删除准重复项，并仅重新获取其副本。 准复制将具有相同的NLP表示，但不一定是完全相同的文本。 （可选）（默认为False）

payload = nucleus_api.TopicContrastModel(dataset=dataset, 
                                         metadata_selection_contrast=metadata_selection_contrast,
                                         num_keywords=num_keywords,
                                         syntax_variables=syntax_variables,
                                         period_start='2018-01-01',
                                         period_end='2018-01-01')
try:
    api_response = api_instance.post_topic_contrast_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    topic_contrast_result = api_response.result
    print('Contrasted Topic')
    print('    Keywords:', topic_contrast_result.keywords)
    print('    Keywords Weight:', topic_contrast_result.keywords_weight)

    print('In-Sample Perf Metrics')
    print('    Accuracy:', topic_contrast_result.perf_metrics.hit_rate)
    print('    Recall:', topic_contrast_result.perf_metrics.recall)
    print('    Precision:', topic_contrast_result.perf_metrics.precision)

### 3.文件分类

此任务需要3个步骤:
1. 使用TopicContrastModel（之前已完成），在带标签的数据集上准备对比主题。


2. 通过提供标记数据集来训练分类器。 
 - 从对比主题调整每个关键字的权重
 - 删除某些关键字
 - 将步骤1生成的对比主题与您自己选择的主题进行比较


3. 测试分类器


在下面的示例中，我们假设已经获得了对比主题（步骤1）。 'fixed_topics'的结构正是Contrasted Topic API的结构

In [ ]:
# 在这里，我们重新使用第2节中的对比主题
fixed_topics = {"keywords": topic_contrast_result.keywords, 
                "weights": topic_contrast_result.keywords_weight} # dict | 对比主题用于分隔两类文件。 重量可选

metadata_selection_contrast = {"label": ["Violence", "All clear"]} # dict | 一个文档可以通过元数据分类成两类文件

query = '' # str | Dataset-language-specific 全文查询，使用mysql MATCH boolean query格式（可选）
custom_stop_words = [""] # 停用词列表。 （可选）
excluded_docs = '' # str | 应从分析中排除的文档ID列表。 例如，[“docid1”，“docid2”，...，“docidN”]（可选）
syntax_variables = False # bool | 如果为True，则分类器将在内容变量之上包含与语法相关的变量（可选）（默认为False）
threshold = 0 # float | Threshold value for a document exposure to the contrasted topic, above which the document is assigned to class 1 specified through metadata_selection. (optional) (default to 0)
remove_redundancies = False # bool | 如果为True，则此选项从分析中删除准重复项，并仅重新获取其副本。 准复制将具有相同的NLP表示，但不一定是完全相同的文本。 （可选）（默认为False）


payload = nucleus_api.DocClassifyModel(dataset=dataset,
                                        fixed_topics=fixed_topics,
                                        metadata_selection_contrast=metadata_selection_contrast,
                                        validation_phase=True,
                                        syntax_variables = syntax_variables,
                                        period_start='2019-01-01',
                                        period_end='2019-01-01')

try:
    api_response = api_instance.post_doc_classify_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    print('Detailed Results')
    print('    Docids:', api_response.result.detailed_results.docids)
    print('    Exposure:', api_response.result.detailed_results.exposures)
    print('    Estimated Category:', api_response.result.detailed_results.estimated_class)
    print('    Actual Category:', api_response.result.detailed_results.true_class)
    print('\n')

    print('Out-Sample Perf Metrics')
    print('    Accuracy:', api_response.result.perf_metrics.hit_rate)
    print('    Recall:', api_response.result.perf_metrics.recall)
    print('    Precision:', api_response.result.perf_metrics.precision)

然后，我们可以进入测试阶段

In [ ]:
payload = nucleus_api.DocClassifyModel(dataset=dataset,
                                        fixed_topics=fixed_topics,
                                        metadata_selection_contrast=metadata_selection_contrast,
                                        custom_stop_words=custom_stop_words,
                                        validation_phase=False,
                                        period_start='2019-01-01',
                                        period_end='2019-01-01')

try:
    api_response = api_instance.post_doc_classify_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    print('Detailed Results')
    print('    Docids:', api_response.result.detailed_results.docids)
    print('    Exposure:', api_response.result.detailed_results.exposures)
    print('    Estimated Category:', api_response.result.detailed_results.estimated_class)

### 4.	微调

#### a. 对比主题指定的`metadata_selection_contrast`

-     对比包含不同关键字的文档

    这对于检测在特定上下文中频繁出现的某些表达式非常有用

In [ ]:
metadata_selection_contrast = {"content": "kill hate torture"}

-     对比来自不同作者的文件

    这对于检测链接到同一实际人员的多个帐户非常有用

In [ ]:
metadata_selection_contrast = {"author": "@suspicious_author"}

#### b.	降低低质量内容检测中的噪音

-   通过使用Contrast Analysis API中的`custom_stop_words`参数，排除无关或次要关键字/主题以定制内容分类


-	提取语料库中文档的关键主题并打印这些主题的关键字


In [ ]:
print('------------- Get list of topics from dataset --------------')

payload = nucleus_api.Topics(dataset=dataset,                         
                            query='',                       
                            num_topics=20, 
                            num_keywords=8,
                            metadata_selection=metadata_selection_contrast)

try:
    api_response = api_instance.post_topic_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    for i, res in enumerate(api_response.result.topics):
        print('Topic', i, ' keywords: ', res.keywords)    
        print('---------------')

使用您的专业知识，您可以确定特定主题或关键字是否没有足够的差异，从而导致低质量的内容检测。


然后，您可以通过创建包含这些单词的`custom_stop_words`变量来定制低质量内容检测。 如下所示，将初始化变量传递到第2节主代码的有效负载中：

In [ ]:
custom_stop_words = ["tough dude","bad boy"] # str | List of stop words. (optional)

#### c. 将内容检测重点放在您的语料库中可能讨论的特定主题上
**query**: 您可以通过利用Contrasted Topic和Document Classify API的`query`变量来优化内容检测。

从您的语库中提及特定主题的内容中重新运行这两个API中的任何一个。 创建变量查询并将其传递给有效负载：

In [ ]:
query = '(LOL OR league of legends OR WOW OR world of warcraft)' # str | Fulltext query, using mysql MATCH boolean query format. Example: "(word1 OR word2) AND (word3 OR word4)" (optional)

版权(c) 2019年 SumUp Analytics 公司 版权所有。 

通知：所有信息均属于SumUp Analytics Inc公司及其供应商的财产。 本合同所包含的知识产权和技术概念属于SumUp Analytics Inc.及其供应商的专利，可由美国和外国专利、在工艺中的专利以及受贸易秘密或版权法保护的专利涵盖。 

除非得到SumUp Analytics公司的事先书面批准，否则严禁传播此类信息或复制此材料。 